In [180]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.pipeline import make_pipeline
import xgboost as xgb
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer, log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import fbeta_score

from sklearn import metrics
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import GroupShuffleSplit

In [74]:
categorical_ftrs = ['Prscrbr_City',
                    'Prscrbr_State_Abrvtn',
                    'Brnd_Name',
                    'Gnrc_Name']

std_ftrs = ['Tot_Clms', 
            'Tot_30day_Fills', 
            'Tot_Day_Suply', 
            'Tot_Drug_Cst', 
            'Tot_Benes', 
            'GE65_Tot_Clms',
            'GE65_Tot_30day_Fills',
            'GE65_Tot_Drug_Cst',
            'GE65_Tot_Day_Suply',
            'GE65_Tot_Benes']



#clf = Pipeline(steps=[('preprocessor', preprocessor)])                                               


In [21]:
X_subsample2

,Prscrbr_City,Prscrbr_State_Abrvtn,Brnd_Name,Gnrc_Name,Tot_Clms,Tot_30day_Fills,Tot_Day_Suply,Tot_Drug_Cst,Tot_Benes,GE65_Tot_Clms,GE65_Tot_30day_Fills,GE65_Tot_Drug_Cst,GE65_Tot_Day_Suply,GE65_Tot_Benes
0,Seattle,WA,Aripiprazole,Aripiprazole,24,24.0,718,1198.86,NaN,0.0,0.0,0.00,0.0,NaN
1,Seattle,WA,Bupropion Xl,Bupropion Hcl,21,33.0,990,538.93,NaN,NaN,NaN,NaN,NaN,NaN
2,Seattle,WA,Clonazepam,Clonazepam,27,31.0,806,284.75,NaN,27.0,31.0,284.75,806.0,NaN
3,Seattle,WA,Clonidine Hcl,Clonidine Hcl,13,17.1,514,78.13,NaN,NaN,NaN,NaN,NaN,NaN
4,Seattle,WA,Duloxetine Hcl,Duloxetine Hcl,13,13.0,390,477.59,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4626,Boca Raton,FL,Vyvanse,Lisdexamfetamine Dimesylate,18,18.0,540,5933.52,NaN,0.0,0.0,0.00,0.0,NaN
4627,Boca Raton,FL,Ziprasidone Hcl,Ziprasidone Hcl,49,59.0,1770,2619.93,NaN,NaN,NaN,NaN,NaN,NaN
4628,Boca Raton,FL,Zolpidem Tartrate,Zolpidem Tartrate,158,183.0,5490,1199.12,22.0,127.0,146.0,1057.17,4380.0,NaN
4629,Tampa,FL,Estradiol,Estradiol,17,23.0,664,1930.07,NaN,17.0,23.0,1930.07,664.0,NaN


In [119]:
param_grid = {"xgbclassifier__subsample": [0.5, 0.7, 0.9],
              "xgbclassifier__missing": [np.nan],
              "xgbclassifier__max_depth": [1, 3, 10],
              "xgbclassifier__learning_rate": [0.001, 0.01, 0.1, 0.3],
              "xgbclassifier__n_estimators": [1000],
              "xgbclassifier__gamma": [1,5,10]}


param_grid1 = {
              "xgbclassifier__missing": [np.nan],
              "xgbclassifier__learning_rate": [0.2],
              "xgbclassifier__max_depth": [15],
              "xgbclassifier__gamma": [5],
              "xgbclassifier__n_estimators": [300]}


fit_params = {"xgbclassifier__early_stopping_rounds": 50}


In [181]:
X = pd.read_csv('X_3specialties_equalWeight_subsample.zip',compression='zip', index_col=False)
y = pd.read_csv('y_3specialties_equalWeight_subsample.zip',compression='zip')
groups = pd.read_csv('groups_3specialties_equalWeight_subsample.zip',compression='zip')

X = X.iloc[:,1:]
y = y.iloc[:,1:]
groups = groups.iloc[:,1:]

y_columns = y.columns

le = LabelEncoder()
y = y.values.ravel()
y = le.fit_transform(y)
y = pd.DataFrame(y)
y.columns = y_columns

In [182]:
X.shape


(5025, 14)

### XGBoost with GridSearchCV (no early stopping)

In [199]:
def ML_pipeline_groups_GridSearchCV(X,y,groups,random_state,n_folds):
    # create a test set based on groups
    splitter = GroupShuffleSplit(n_splits=1,test_size=0.2,random_state=random_state)
    
    for i_other,i_test in splitter.split(X, y, groups):
        X_other, y_other, groups_other = X.iloc[i_other], y.iloc[i_other], groups.iloc[i_other]
        X_test, y_test, groups_test = X.iloc[i_test], y.iloc[i_test], groups.iloc[i_test]
        

    # check the split
#     print(pd.unique(groups))
#     print(pd.unique(groups_other))
#     print(pd.unique(groups_test))
    # splitter for _other
    kf = GroupKFold(n_splits=n_folds)
    # create the pipeline: preprocessor + supervised ML method

    
    clf = xgb.XGBClassifier(num_class=3,
                                eval_metric = "mlogloss",
                                objective = "multi:softprob",
                                random_state = random_state, 
                                use_label_encoder = False)
    
    preprocessor = ColumnTransformer(
        transformers=[
        ('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'), categorical_ftrs),
        ('std', StandardScaler(), std_ftrs)])
    
    pipe = make_pipeline(preprocessor,clf)
    
    # the parameter(s) we want to tune
    param_grid = {"xgbclassifier__subsample": [0.5, 0.7],
              "xgbclassifier__missing": [np.nan],
              "xgbclassifier__max_depth": [1, 3, 10, 30],
              "xgbclassifier__learning_rate": [0.01, 0.1, 0.3],
              "xgbclassifier__n_estimators": [300],
              #"xgbclassifier__gamma": [1,5,10],
              "xgbclassifier__reg_alpha" : [0e0, 1e-2, 1e-1, 1e0, 1e1, 1e2],
              "xgbclassifier__reg_lambda": [0e0, 1e-2, 1e-1, 1e0, 1e1, 1e2],
              "xgbclassifier__seed": [random_state]  }
    
    # prepare gridsearch
    #grid = GridSearchCV(pipe, param_grid=param_grid,scoring = make_scorer(accuracy_score),
                        #cv=kf, return_train_score = True)
    
    #LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True, labels = sorted(np.unique(y)))
    
    grid = GridSearchCV(pipe, 
                        param_grid=param_grid1,
                        scoring = "f1_macro",
                        cv=kf, 
                        return_train_score = True, 
                        n_jobs=1, 
                        verbose=10)
    
    # do kfold CV on _other
    grid_result = grid.fit(X_other, y_other, groups=groups_other)
    
    print()
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    
    print(f'Best params: {grid.best_params_}')
    
    print(f"mean CV: {means} +/ {stds}")
    
    y_test_pred_proba = grid.predict_proba(X_test)
    
    y_test_pred = grid.predict(X_test)
    
    #score = accuracy_score(y_test,y_test_pred)
    
    #score = f1_score(y_test,y_test_pred, average = "macro")
    
    cm = confusion_matrix(y_test, y_test_pred)
    #confusion_mat.append(cm)
    
    # get the feature importances
    metrics = ["weight", "gain", "cover", "total_gain", "total_cover"]
    importances = []

    for i in metrics:
        booster_score = clf.best_estimator_.get_booster().get_score(importance_type='gain')
        importances.append(booster_score)
        #.get_booster().get_score(importance_type='weight')

    
    
    #logloss = log_loss(y_test, y_test_pred_proba,labels = sorted(np.unique(y)))
    
    print()
    
    f_05_score = fbeta_score(y_test, y_test_pred, beta = 0.5, labels=sorted(np.unique(y)), average='macro')
    
    print(f'F05: {f_05_score}')
    
    
    
    
    
    
    return f_05_score, grid.best_params_, cm, importances



In [200]:
test_scores = []
best_params = []
confusion_mat = []
xgboost_metrics = []


In [201]:
%%time
y_test_pred_prob_list = []

logloss_list = []

nr_states = [1,2,3]

for i in range(3):
    print(f'Random State # {i}')
    print()
    
    score, bestparams, cmat, importances = ML_pipeline_groups_GridSearchCV(X, y, groups, i*42, 5)
    
    #logloss_list.append(result)
    
   
    test_scores.append(score)
    best_params.append(bestparams)
    confusion_mat.append(cmat)
    xgboost_metrics.append(importances)
    #y_test_pred_prob_list.append(y_test_pred_prob)
    
    #print()

    
    #best_params.append(grid.best_params_)
    #print()
    #print('best CV score:',grid.best_score_)
    #print()
    #print('test score:', score)
    
    #print()
    
#print('test logloss:',np.around(np.mean(logloss_list),2),'+/-',np.around(np.std(logloss_list),2))

Random State # 0

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.2, xgbclassifier__max_depth=15, xgbclassifier__missing=nan, xgbclassifier__n_estimators=100
[CV 1/5; 1/1] END xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.2, xgbclassifier__max_depth=15, xgbclassifier__missing=nan, xgbclassifier__n_estimators=100;, score=(train=0.972, test=0.739) total time=  23.9s
[CV 2/5; 1/1] START xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.2, xgbclassifier__max_depth=15, xgbclassifier__missing=nan, xgbclassifier__n_estimators=100
[CV 2/5; 1/1] END xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.2, xgbclassifier__max_depth=15, xgbclassifier__missing=nan, xgbclassifier__n_estimators=100;, score=(train=0.965, test=0.530) total time=  25.7s
[CV 3/5; 1/1] START xgbclassifier__gamma=5, xgbclassifier__learning_rate=0.2, xgbclassifier__max_depth=15, xgbclassifier__missing=nan, xgbclassifier__n_e

KeyError: 'clf'

### XGBoost with ParamdGrid Iteration (WITH early stopping)

In [184]:
def xgb_model(X_train, Y_train, X_CV, y_CV, X_test, y_test, i, verbose=4):

    # make into row vectors to avoid an obnoxious sklearn/xgb warning
    Y_train = np.reshape(np.array(Y_train), (1, -1)).ravel()
    y_CV = np.reshape(np.array(y_CV), (1, -1)).ravel()
    y_test = np.reshape(np.array(y_test), (1, -1)).ravel()

    
    XGB = xgb.XGBClassifier(num_class=3,
                      eval_metric = "mlogloss",
                      objective = "multi:softprob",
                      random_state = i, 
                      use_label_encoder = False)
    
    # find the best parameter set
    param_grid = {"learning_rate": [0.01, 0.1, 0.3],#0.01,
                  "n_estimators": [10000],
                  "seed": [i],
                  "reg_alpha": [0e0, 1e-2, 1e-1, 1e0, 1e1, 1e2],
                  #"reg_lambda": [0e0, 1e-2, 1e-1, 1e0, 1e1, 1e2],
                  "missing": [np.nan], 
                  "max_depth": [1, 3, 10], #1,3,
                  "colsample_bytree": [0.5, 0.7, 0.9],              
                  "subsample": [0.5, 0.7, 0.9]} #, 0.9
    
    

    pg = ParameterGrid(param_grid)

    scores = np.zeros(len(pg))

    #weights = compute_sample_weight(class_weight='balanced', y = Y_train)
    
    for i in range(len(pg)):
        if verbose >= 5:
            print("Param set " + str(i + 1) + " / " + str(len(pg)))
        params = pg[i]
        XGB.set_params(**params)
        eval_set = [(X_CV, y_CV)]
        XGB.fit(X_train, Y_train,
                early_stopping_rounds = 50, eval_set=eval_set, verbose=False)
                #sample_weight = weights)
            
        #y_CV_pred_proba = XGB.predict_proba(X_CV, iteration_range=(0, XGB.best_ntree_limit))
        y_CV_pred = XGB.predict(X_CV, iteration_range=(0, XGB.best_ntree_limit))
        #scores[i] = accuracy_score(y_CV,y_CV_pred)
        scores[i] = fbeta_score(y_CV, y_CV_pred, beta = 0.5, labels=sorted(np.unique(y)), average='macro')

    best_params = np.array(pg)[scores == np.max(scores)]
    
    if verbose >= 4:
        print(f'Validation set best score {np.max(scores)}')
        print()
        print(f'Validation set best params {best_params}')
        print()
        
    # test the model on the test set with best parameter set
    XGB.set_params(**best_params[0])
    
    XGB.fit(X_train,Y_train,
            early_stopping_rounds=50, eval_set=eval_set, verbose=False)
    
    y_test_pred = XGB.predict(X_test, iteration_range=(0, XGB.best_ntree_limit))
    
    #test_logloss = log_loss(y_test, y_test_pred_proba)

    if verbose >= 1:
        print ('The Test F0.5 is:', fbeta_score(y_test, y_test_pred, beta = 0.5, labels=sorted(np.unique(y)), average='macro')) 
        print()
        
    if verbose >= 2:
        #print ('The predictions are:')
        #print (y_test_pred)
        print()
    #if verbose >= 3:
        #print("Feature importances:")
        #print(XGB.feature_importances_)
    
    return (fbeta_score(y_test, y_test_pred, beta = 0.5, labels=sorted(np.unique(y)), average='macro')
            , y_test_pred, XGB.feature_importances_)
    #return (y_test_pred, best_params, XGB.feature_importances_ )


In [185]:
def ML_pipeline_XGBoost_EarlyStop(X,y,groups,random_state,n_folds):
    
    
    result_by_fold = []
    
    prediction_by_fold = []
    
    feat_impo_by_fold = []
    
    # create a test set based on groups
    splitter = GroupShuffleSplit(n_splits=1,test_size=0.2,random_state=random_state)
    
    for i_other,i_test in splitter.split(X, y, groups):
        X_other, y_other, groups_other = X.iloc[i_other], y.iloc[i_other], groups.iloc[i_other]
        X_test, y_test, groups_test = X.iloc[i_test], y.iloc[i_test], groups.iloc[i_test]
        

    splitter = GroupShuffleSplit(n_splits=1,test_size=0.2,random_state=i)
    
    # Get Test Set
    for i_other,i_test in splitter.split(X, y, groups):
        X_other, y_other, groups_other = X.iloc[i_other], y.iloc[i_other], groups.iloc[i_other]
        X_test, y_test, groups_test = X.iloc[i_test], y.iloc[i_test], groups.iloc[i_test]
    
    # Get Validation Set
    
    kf = GroupKFold(n_splits=n_folds)
    counter = 0
    
    for i_train, i_test in kf.split(X_other, y_other, groups_other):
        X_train, y_train, groups_train = X_other.iloc[i_train], y_other.iloc[i_train], groups_other.iloc[i_train]
        X_val, y_val, groups_val = X_other.iloc[i_test], y_other.iloc[i_test], groups_other.iloc[i_test]
        
        #print(len(y_val))
        #print(len(y_train))
        counter = counter + 1
        
        print(f"CV # {counter}")
        X_prep = preprocessor.fit_transform(X_train)
        # collect feature names
        
        feature_names = preprocessor.get_feature_names_out()
        
        df_train = pd.DataFrame(data=X_prep,columns=feature_names)
        print(f"Train Set Shape (after preprocessing): {df_train.shape}")
        print()
        
        # transform the CV
        df_CV = preprocessor.transform(X_val)
        df_CV = pd.DataFrame(data=df_CV,columns = feature_names)
        print(f"CV Set Shape (after preprocessing): {df_CV.shape}")
        print()
        # transform the test
        df_test = preprocessor.transform(X_test)
        df_test = pd.DataFrame(data=df_test,columns = feature_names)
        print(f"Test Set Shape (after preprocessing): {df_test.shape}")
        print()
        
        y_CV = y_val
        
        
        # run XGB
        result, prediction, featimpo = xgb_model(df_train, y_train, df_CV, y_CV, df_test, y_test, i, verbose=4)
        
        #feat_impo.append(sub_y_test_pred[2])
        
        #sub_y_test_pred = pd.DataFrame(sub_y_test_pred[1],columns=['y_test_pred'])
        
    
        result_by_fold.append(result)
        
        prediction_by_fold.append(prediction)
        
        feat_impo_by_fold.append(featimpo)

        # get global scores
        #total_accuracy = (accuracy_score(y_test,sub_y_test_pred))
        #f1 = f1_score(y_test,sub_y_test_pred, average = 'macro')

        #cm = confusion_matrix(y_test,sub_y_test_pred)
        
        #print()
        #print('   Test Accuracy:', accuracy_score(y_test,sub_y_test_pred))
        #print('   Test F1 Score:', f1_score(y_test, sub_y_test_pred, average = 'macro'))

        
    return (result_by_fold, prediction_by_fold, feat_impo_by_fold)

In [176]:
all_y_test_pred = []
feat_impo_list = []
test_scores = []
best_params = []
confusion_mat = []
class_met = []

In [183]:
%%time

test_scores = []

for i in range(3):
    print(f'Random State # {i}')
    print()
    
    result, prediction, featimpo = ML_pipeline_XGBoost_EarlyStop(X, y, groups, i*42, 5)
    
    #confusion_mat.append(cm)
    
    #feat_impo_list.append(feat_impo)
    
    #class_met.append(class_metrics)
    print()
    #print(f'Best params: {grid.best_params_}')
    
    #best_params.append(grid.best_params_)
    #print()
    #print('best CV score:',grid.best_score_)
    #print()
    #print('test score:', score)
    test_scores.append(result)
    print()
    
print('test accuracy:',np.around(np.mean(test_scores),2),'+/-',np.around(np.std(test_scores),2))

Random State # 0

CV # 1
Train Set Shape (after preprocessing): (3198, 906)

CV Set Shape (after preprocessing): (800, 906)

Test Set Shape (after preprocessing): (1027, 906)

Validation set best score and parameters are:
0.7036231770712799
[{'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 3, 'missing': nan, 'n_estimators': 10000, 'seed': 0, 'subsample': 0.7}]

The Test F0.5 is: 0.6851974127939119


CV # 2
Train Set Shape (after preprocessing): (3198, 918)

CV Set Shape (after preprocessing): (800, 918)

Test Set Shape (after preprocessing): (1027, 918)



KeyboardInterrupt: 